In [6]:
import nibabel as nib
import numpy as np
import os
import glob
import SimpleITK as sitk

In [4]:
base_dir = r"../DataSet"
updated_dir = r"../Updated-DataSet"

# Use glob to find all files in the directory
file_paths = glob.glob(os.path.join(base_dir, '*'))

# Loop through each file and construct the file path
for file_path in file_paths:
    # Extract the file name from the file path
    file_name = os.path.basename(file_path)
    # Load the nifti file
    nib_file = nib.load(file_path)
    # Extract the header from the nifti file
    header = nib_file.header
    # Extract the voxel spacing from the header
    voxel_spacing = header.get_zooms()
    # Extract the qoffsets from the header
    qoffset_x = header['qoffset_x']
    qoffset_y = header['qoffset_y']
    qoffset_z = header['qoffset_z']
    # Create a new affine matrix
    new_affine = np.array([
    [voxel_spacing[0], 0, 0, qoffset_x],
    [0, voxel_spacing[1], 0, qoffset_y],
    [0, 0, voxel_spacing[2], qoffset_z],
    [0, 0, 0, 1]
    ])

    # Override the affine matrix in the header
    new_header = header.copy()

    new_header['srow_x'] = new_affine[0,:]
    new_header['srow_y'] = new_affine[1,:]
    new_header['srow_z'] = new_affine[2,:]


    # Create new file using the updated affine matrix
    updated_img = nib.Nifti1Image(nib_file.get_fdata(), new_affine, new_header)
    nib.save(updated_img, os.path.join(updated_dir, "updated_" + file_name))




In [5]:
updated_dir = r"../Updated-DataSet"
downsampled_dir = r"../Downsampled-DataSet"

# Use glob to find all files in the directory
file_paths = glob.glob(os.path.join(updated_dir, '*'))

# Loop through each file and construct the file path
for file_path in file_paths:
    # Extract the file name from the file path
    file_name = os.path.basename(file_path)
    # Load the nifti file
    nib_file = nib.load(file_path)

    # Downsample the images
    downsampled_img = nib_file.slicer[::5,:,:]

    # Save the downsampled images
    nib.save(downsampled_img, os.path.join(downsampled_dir, "downsampled_" + file_name))

In [7]:
def resample_volume(volumne_path="", interpolator=sitk.sitkLinear, new_spacing=None, output_path=""):
    if new_spacing is None:
        new_spacing = [1, 1, 1]

    voulume = sitk.ReadImage(volumne_path, sitk.sitkFloat32)
    original_spacing = voulume.GetSpacing()
    original_size = voulume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)]
    resample_volume = sitk.Resample(voulume, new_size, sitk.Transform(), interpolator, voulume.GetOrigin(), new_spacing, voulume.GetDirection(), 0.0, voulume.GetPixelID())

    sitk.WriteImage(resample_volume, output_path)

In [8]:
# Resample the updated files
input_dir = r"../Updated-DataSet"
output_dir = r"../Resampled-DataSet/Original"

# Use glob to find all files in the directory
file_paths = glob.glob(os.path.join(input_dir, '*'))

# Loop through each file and construct the file path
for file_path in file_paths:
    # Extract the file name from the file path
    file_name = os.path.basename(file_path)
    # Resample the images
    resample_volume(file_path, output_path=os.path.join(output_dir, file_name))

In [11]:
# Resample the updated files
input_dir = r"../Downsampled-DataSet"
output_dir = r"../Resampled-DataSet/Downsampled"

# Use glob to find all files in the directory
file_paths = glob.glob(os.path.join(input_dir, '*'))

# Loop through each file and construct the file path
for file_path in file_paths:
    # Extract the file name from the file path
    file_name = os.path.basename(file_path)
    # Resample the images
    resample_volume(file_path, output_path=os.path.join(output_dir, file_name))